# Env Troubleshooting Report

> This notebook is a bare bones snippet from the main environment testing notebooks.
- **Ideally, you should first open and run all of the cells in the appropriate environment tester notebook** (see list of file names below)
- In the main folder of this repo (above this Troubleshooting folder):
	- "EnvironmentTester-mac.ipynb" 
	- "EnvironmentTester-windows.ipynb"
	

- You can also just run the first code cells in this notebook to get a barebones report about your environment.

> - **You should show the output of both notebooks to your instructor or TA when troubleshooting.**




### Set the Output Folder And Delete Previous .txt files

In [ ]:
from IPython.display import clear_output
try:
    from tzlocal import get_localzone, get_localzone_name
except:
    !pip install tzlocal
    from tzlocal import get_localzone, get_localzone_name
    

In [ ]:
import os, glob, sys
VERBOSE = False

## Saving kernel's current location to cd back to at end
start_dir = os.path.abspath(os.curdir)

## Make sure troubleshooting folder exists
filepath = os.path.abspath(os.path.curdir)
if filepath.endswith('Troubleshooting') == False:
	os.makedirs('Troubleshooting',exist_ok=True)
	output_folder = os.path.abspath('./Troubleshooting/')
else:
	output_folder = os.path.abspath(os.path.curdir)
	# relative_folder = os.path.curdir()

## Normalize file path for Windows compatibility
output_folder = os.path.normpath(output_folder)
if VERBOSE:
	print(f'- The report will be saved in this folder: "{output_folder}"')
    
## Change kernel to output_folder
os.chdir(output_folder)

In [ ]:
## check if txt files already exist
 
## if so, delete the files (list comprehensions way)
txt_files = glob.glob(output_folder+"/temp*.txt")
[os.remove(f) for f in txt_files]		

## Confirm they're deleted		
txt_files = glob.glob(output_folder+"/temp*.txt")

if VERBOSE:
	if len(txt_files) == 0:
		print('- [i] Previous text files successfully deleted.')
	else:
		print(f'[!] Previous text files not fully delted ({len(txt_files)} files remain)')

### Save Environment Info from Terminal Commands

In [ ]:
## Change kernel to output_folder
os.chdir(output_folder)

In [ ]:
import sys

def get_platform():
    """SOURCE: https://www.webucator.com/article/how-to-check-the-operating-system-with-python/"""
    platforms = {
        'linux1' : 'Linux',
        'linux2' : 'Linux',
        'darwin' : 'OS X',
        'win32' : 'Windows'
    }
    if sys.platform not in platforms:
        return sys.platform
    
    return platforms[sys.platform]

platform = get_platform()
with open('temp-operating system.txt','w') as f:
    f.write(f"Platform: {platform}\n")

In [ ]:
# ## Exporting Troubleshooting reports 

import os,glob
os.system('echo $PATH >> temp-shell_path.txt')
os.system('conda info >> temp-conda_info.txt')
os.system('conda env list >> temp-conda_env_list.txt')
os.system('conda list >> temp-conda_list.txt')
os.system('jupyter kernelspec list >> temp-jupyter_kernelspec_list.txt')
os.system('which python >> temp-which_python.txt')
os.system('whoami >> temp-whoami.txt')

# Added 08/08/23
!echo "Testing running shell commands from within jupyter.">> "temp-echo_test.txt"

## get all .txt files
txt_files = sorted(glob.glob(output_folder+"/temp*.txt"))
len(txt_files)


In [ ]:
# ## test getting simple name from fpath
# os.path.basename(txt_files[0])

### Combine Text File Outputs into 1 Report

In [ ]:
## Combine Text Files into a Dictionary with filename as the keyd
raw_report = {}
for file in txt_files:
	## simple filename for key
	simple_fname = os.path.basename(file).replace('.txt','').replace('temp-','') # added temp 080823
	
	# Open file and save its contents into dict
	with open(file) as f:
		raw_report[simple_fname] =f.read()
		
raw_report.keys()

In [ ]:
## Setting Preferred Order (conda list last) using orderd_report list
if "conda list" in raw_report.keys():
	conda_list = raw_report.pop('conda list')
else:
    conda_list = "- ERROR USING !conda list command: this machine may not be able to display bash outputs in jupyter notebooks.\n"

if "echo_test" in raw_report.keys():
	echo = raw_report.pop('echo_test')
else:
    echo = "- ERROR USING !echo command: this machine may not be able to display bash outputs in jupyter notebooks.\n"
    
# Make the ordered report    
ordered_report = []
for key, data in raw_report.items():
	ordered_report.append({key:data})
ordered_report.append({'conda list':conda_list})
ordered_report.append({'echo test':echo})

In [ ]:
# ## Adding echo test to report (08/08/23)
# if "echo_test" in raw_report.keys():
# 	echo = raw_report.pop('echo_test')
# else:
#     echo = "- ERROR USING !echo command: this machine may not be able to display bash outputs in jupyter notebooks.\n"
# ordered_report.append({'Echo Test':echo})

In [ ]:
## Generating final report 
report_list = []
banner = "===="*20

for report_dict in ordered_report:
	for name, txt_report in report_dict.items():
		
		## Start report piece with banner
		report_piece = f"{banner}\n- [i] {name}:\n{banner}\n"
		
		## Split System Path for readability
		if name == "shell_path":
			txt_report = txt_report.replace(":","\n:")
			
		## Append file's contents
		report_piece += txt_report
		report_piece+=f"{banner}\n\n"
		
		report_list.append(report_piece)

## Deleting individual text files before saving combined file
txt_files = glob.glob(output_folder+"/*.txt")		
[os.remove(f) for f in txt_files];


In [ ]:
## Recording time notebook was run
import datetime as dt
from tzlocal import get_localzone, get_localzone_name
now = dt.datetime.now(get_localzone())
now_nice = now.strftime("%m/%d/%Y @ %I:%M:%S %p ") + f"(tz={get_localzone_name().split('/')[-1]})"
now_nice

In [ ]:
os.path.normpath(os.path.join(output_folder , "FINAL_REPORT.txt"))

In [ ]:
## Saving as FINAL REPORT.txt
report_fname = os.path.normpath(os.path.join(output_folder , "FINAL_REPORT.txt"))
with open(report_fname,'w') as f:
    final_txt = "\n".join(report_list)
    final_txt += banner
    final_txt += f"\n - Report created at: {now_nice}"
    final_txt += '\n'+banner
    f.write(final_txt)

In [ ]:
## Getting relative filepath for display in env testing notebooks
try:
    report_rel_fname= "dojo-env-setup" + report_fname.split(os.path.normpath("dojo-env-setup/"))[1]
except:
    report_rel_fname = report_fname
print(f'- [i] Final report saved as {report_rel_fname}')


In [ ]:
with open(report_fname) as f:
	final_report = f.read()


if VERBOSE:
	print('- [i] FINAL REPORT:\n')
	print(final_report)
else:
	print('- [i] Short Preview of FINAL REPORT:\n')
	print(final_report[:1000])
# print(final_report)

In [ ]:
## REMOVING TEMP FILSE 
txt_files = glob.glob(output_folder+"/temp*.txt")
[os.remove(f) for f in txt_files]		
txt_files = glob.glob(output_folder+"/temp*.txt")
len(txt_files)

In [ ]:
# Changing back to original directory
os.chdir(start_dir)